## 2.11

Find the fixed point for $q_{k + 1} = 1 + (1 - x) q_k$. At the fixed point

$ q_{\infty} = 1 + (1 - x) q_{\infty}$

Rearrange to get

$ q_{\infty} = \frac{1}{x} $

Second part

$ Q_k = q_k ( 1 - y ) $

$ \implies Q_{k + 1} = 1 + (1 - (1 - y)) q_{k} (1 - y) $

$ \implies Q_{k + 1} = 1 + y Q_k $

Show the series

$ Q_1 = 1 + y$ whern $Q_0 = 1$

$ Q_2 = 1 + y + y^2 $ 

Assume

$ Q_k = \sum_{i=0}^{k}{y^i} $

Then $Q_{k+1} = 1 + y Q_k = \sum_{i=0}^{k + 1}{y^i}$

by indicution we have the series. Now by doing the standard error analysis, we have

$e_k = y^k e_0$

where $e_k = Q_k - Q_{\infty}$. Clearly $|y| < 1$ to converge.

## 2.12

Same gain as before, subtract $u_{\infty}$ from both sides 

$ u_{k+1} = u_k + p(A) (b - A u_k) $

$ \implies u_{k + 1} - u_{\infty} = u_k - u_{\infty} + p(A) (b - A u_k) $

$ \implies e_{k + 1} = u_k - \left[ u_{\infty} + p (b - A u_{\infty}) \right] + p(A) (b - A u_k) $

$ \implies e_{k + 1} = e_k ( 1- p(A) A) $

$ \implies e_{k + 1} = q_k e_k $

using the definition give in the text for $q$ 

In [7]:
import numpy as np

In [2]:
A = numpy.array([[1, 2, 3, 0], [2, 1, -2, -3], [-1, 1, 1, 0], [0, 1, 1, -1]], dtype=float)
b = numpy.array([7, 1, 1, 3], dtype=float)

In [3]:
numpy.linalg.inv(A).dot(b)

array([ 1.,  0.,  2., -1.])

## 2.13

```
16d15
<     PetscMPIInt rank;
20,21d18
<     PetscCall(MPI_Comm_rank(PETSC_COMM_WORLD,&rank));
< 
25,29c22
< 
<     if (rank == 0) {
<       PetscCall(VecSetValues(b,4,j,ab,INSERT_VALUES));
<     }
< 
---
>     PetscCall(VecSetValues(b,4,j,ab,INSERT_VALUES));
31a25
> 
36,38c30
< 
<     if (rank == 0) {
<       for (i=0; i<4; i++) {   // set entries one row at a time
---
>     for (i=0; i<4; i++) {   // set entries one row at a time
40d31
<       }

```

Works in parallel and in serial

```
$ mpiexec -n 2 ./vmkrankzero                                                                                                                         
Vec Object: 2 MPI processes 
  type: mpi               
Process [0]               
1.                         
-7.77156e-16                    
Process [1]
2.
-1.                                    
```


## 2.14

yes, it does diverge even with 4x4, `./tri -tri_m 4 -ksp_monitor -ksp_type richardson -pc_type none` for example

## 2.15

It gets larger only very slowly for cholesky

|  Size  | error |
|--------|-------|
| 4      | 6.7e-16 |
| 100    | 3.2e-15 |
| 1000   | 7.5e-15 |
| 5000   | 1.8e-14  |
| 100000 | 8.6e-14 |
| 1000000 | 2.6e-13 |

In theory the error shouldn't get that much worse with direct methods. In practice there is some accumulation of round off errror. 

## 2.16

In [80]:
def gauss(aij, b):
    aij = aij.copy()
    b = b.copy()
    n = len(aij)
    for i in range(1, n):
        ratio = aij[i, i - 1] / aij[i - 1, i - 1]
        for j in range(2):
            aij[i, i + j - 1] = aij[i, i + j - 1] - aij[i - 1, i + j - 1] * ratio
        b[i] = b[i] - b[i - 1] * ratio

    x = np.zeros(n, dtype=float)
    x[n - 1] = b[n - 1] / aij[n - 1, n - 1]
    for i in range(n - 1)[::-1]:
        x[i] = (b[i] - x[i + 1] * aij[i, i + 1]) / aij[i , i]
    return x

In [81]:
def generate(n):
    a = np.random.random(n)
    b = np.random.random(n - 1)
    c = np.random.random(n - 1)
    return (np.diag(a, 0) + np.diag(b, -1) + np.diag(c, 1), np.random.random(n))

In [87]:
def error(aij, x, b):
    return np.linalg.norm(np.dot(aij, x) - b, ord=2)

In [92]:
aij, b = generate(10)

In [94]:
x = gauss(aij, b)

In [95]:
error(aij, x, b)

5.900916318210353e-16

From the above algorithm the flop count is

 - N - 1 divides
 - (N - 1) * 2 subs
 - (N - 1) * 2 muls
 - N subs
 - N muls
 - 1 div
 - N - 1 divs
 - N - 1 subs
 - N - 1 muls

This seems to give

10 * N - 7 ????

## 2.17

GMRES solves tridiagonal in one shot

`./tri -tri_m 10000 -ksp_monitor -ksp_type gmres`

For toeplitz + tridiagonal the formula for eigenvalues is

$a + 2 \sqrt(b c) \cos \left( \frac{ k \pi }{n  + 1} \right) $

It's spot on for `N = 10`. For higher values of N it's giving a subset and it's not as accurate?
The maximum and minimum eigenvalues are bounded by 5 and 1 which is quite a small ratio so condition number is good

In [100]:
n = 100
a, b, c = 3.0, -1.0, -1.0
k = np.arange(n) + 1
eigen = a + 2 * np.sqrt(b * c) * np.cos( k * np.pi / (n + 1)) 

In [101]:
eigen

array([4.99903256, 4.99613119, 4.9912987 , 4.98453974, 4.97586088,
       4.9652705 , 4.95277884, 4.938398  , 4.92214188, 4.90402622,
       4.88406853, 4.86228812, 4.83870608, 4.8133452 , 4.78623003,
       4.7573868 , 4.72684341, 4.69462941, 4.66077597, 4.62531583,
       4.5882833 , 4.54971421, 4.50964588, 4.46811706, 4.42516793,
       4.38084004, 4.33517628, 4.28822082, 4.24001909, 4.19061773,
       4.14006452, 4.08840837, 4.03569925, 3.98198816, 3.92732706,
       3.87176884, 3.81536723, 3.75817681, 3.7002529 , 3.64165154,
       3.58242942, 3.52264385, 3.46235264, 3.40161415, 3.34048711,
       3.27903068, 3.2173043 , 3.15536769, 3.09328078, 3.03110362,
       2.96889638, 2.90671922, 2.84463231, 2.7826957 , 2.72096932,
       2.65951289, 2.59838585, 2.53764736, 2.47735615, 2.41757058,
       2.35834846, 2.2997471 , 2.24182319, 2.18463277, 2.12823116,
       2.07267294, 2.01801184, 1.96430075, 1.91159163, 1.85993548,
       1.80938227, 1.75998091, 1.71177918, 1.66482372, 1.61915

## 2.18

 - LU doesn't work in parallel
 - neither does cholesky
 - ilu doesn't work in parallel
 - gmres:bjacobi+ilu on one process works for me, maybe I have a lot of memory on my laptop. Alos, the real time was 28s and the sys time was 1.743 so might be very inefficient on one process
 - icc obviously isn't parallel capable
 - no need to run bjacobi+icc as icc works alone on a single process

## 2.19

|  KSP | PC |  P=1  | P=4 |
|------|----|-------|-----|
| preonly | lu | 1 |   |
|         | cholesky | 1 |   |
| richardson | jacobi | 56 |  56 |
| gmres | none | 17 | 17 |
|       | jacobi | 17 | 17 |



## 2.20

This just gave an error

## 2.21

Made the following diff to make this work

```
diff --git a/c/ch2/tri.c b/c/ch2/tri.c
index 570d18e..97a7031 100644
--- a/c/ch2/tri.c
+++ b/c/ch2/tri.c
@@ -45,12 +45,14 @@ int main(int argc,char **args) {
         }
         xval = PetscExpReal(PetscCosReal((double)i));
         PetscCall(VecSetValues(xexact,1,&i,&xval,INSERT_VALUES));
+        xval = 0.0;
+        PetscCall(VecSetValues(b,1,&i,&xval,INSERT_VALUES));
     }
     PetscCall(MatAssemblyBegin(A,MAT_FINAL_ASSEMBLY));
     PetscCall(MatAssemblyEnd(A,MAT_FINAL_ASSEMBLY));
     PetscCall(VecAssemblyBegin(xexact));
     PetscCall(VecAssemblyEnd(xexact));
-    PetscCall(MatMult(A,xexact,b));
+    //    PetscCall(MatMult(A,xexact,b));
 
     PetscCall(KSPCreate(PETSC_COMM_WORLD,&ksp));
     PetscCall(KSPSetOperators(ksp,A,A));


```

## 2.22

Don't have valgrind installed